In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-07 12:01:44--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.93MB/s    in 0.2s    

2023-01-07 12:01:44 (5.93 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
# Start spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ETL_final_project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# import libraries for Exploratory Data Analysis
import pandas as pd
from pyspark import SparkFiles
import numpy as no
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sn

In [5]:
# Drive helper
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
path ="/content/drive/MyDrive/data/Daily_County-Level_Ozone_Concentrations__2001-2016.csv"
ozone_df = pd.read_csv(path)
ozone_df.tail()


,Year,Month,Day,statefips,countyfips,O3_max_pred,O3_med_pred,O3_mean_pred,O3_pop_pred
18168991,2016,DEC,31,56,37,41.8023,40.98200,40.954517,41.030787
18168992,2016,DEC,31,56,39,39.8256,39.24925,39.252300,39.160351
18168993,2016,DEC,31,56,41,42.7244,39.59090,40.449267,40.374935
18168994,2016,DEC,31,56,43,35.3622,33.30220,33.924567,34.047683
18168995,2016,DEC,31,56,45,39.6921,38.58465,38.584650,38.673762


In [7]:
path2="/content/drive/MyDrive/data/cancer_incidence.csv"
df_cancer = pd.read_csv(path2)
pd.set_option('display.max_columns', None)
df_cancer.head()

,index,County,FIPS,"Age-Adjusted Incidence Rate(Ê) - cases per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,Average Annual Count,Recent Trend,Recent 5-Year Trend (ˆ) in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1
0,0,"US (SEER+NPCR)(1,10)",0,62.4,62.3,62.6,214614,falling,-2.5,-3,-2
1,1,"Autauga County, Alabama(6,10)",1001,74.9,65.1,85.7,43,stable,0.5,-14.9,18.6
2,2,"Baldwin County, Alabama(6,10)",1003,66.9,62.4,71.7,170,stable,3,-10.2,18.3
3,3,"Barbour County, Alabama(6,10)",1005,74.6,61.8,89.4,25,stable,-6.4,-18.3,7.3
4,4,"Bibb County, Alabama(6,10)",1007,86.4,71,104.2,23,stable,-4.5,-31.4,32.9


In [8]:
#get numbber of rows and columns
ozone_df.shape

(18168996, 9)

In [9]:
df_cancer.shape

(3141, 11)

In [10]:
df_cancer.dtypes

index                                                  int64
County                                                object
 FIPS                                                  int64
Age-Adjusted Incidence Rate(Ê) - cases per 100,000    object
Lower 95% Confidence Interval                         object
Upper 95% Confidence Interval                         object
Average Annual Count                                  object
Recent Trend                                          object
Recent 5-Year Trend (ˆ) in Incidence Rates            object
Lower 95% Confidence Interval.1                       object
Upper 95% Confidence Interval.1                       object
dtype: object

In [11]:
df_cancer['Recent Trend'].value_counts()

stable     2429
*           260
falling     200
_           192
rising       43
__           17
Name: Recent Trend, dtype: int64

In [12]:
df_cancer.isnull().sum().values.any()

False

In [13]:
# has special characters
print(df_cancer[df_cancer['Recent Trend'].str.contains(r'[*_―—-]')])

      index                                    County   FIPS  \
68       68      Aleutians East Borough, Alaska(6,10)   2013   
69       69  Aleutians West Census Area, Alaska(6,10)   2016   
72       72         Bristol Bay Borough, Alaska(6,10)   2060   
73       73                      Denali, Alaska(6,10)   2068   
74       74      Dillingham Census Area, Alaska(6,10)   2070   
...     ...                                       ...    ...   
2930   2930            Lexington City, Virginia(6,10)  51678   
2933   2933        Manassas Park City, Virginia(6,10)  51685   
2963   2963         Garfield County, Washington(6,10)  53023   
3131   3131            Niobrara County, Wyoming(6,10)  56027   
3135   3135            Sublette County, Wyoming(6,10)  56035   

     Age-Adjusted Incidence Rate(Ê) - cases per 100,000  \
68                                                   *    
69                                                   *    
72                                                   *

In [14]:
# drop rows with special characters
df_cancer = df_cancer.drop(df_cancer[df_cancer['Recent Trend'].str.contains(r'[*_―—-]')].index)

In [15]:
df_cancer['Recent Trend'].value_counts()

stable     2429
falling     200
rising       43
Name: Recent Trend, dtype: int64

In [16]:
ozone_df.isnull().values.any()

False

In [17]:
# Remove years after 2014 to match cancer data
ozone_df= ozone_df[ozone_df['Year']<=2014]


In [18]:
# Create FIPS joining countyfips and statefips
ozone_df = ozone_df.astype({'statefips':'string', 'countyfips':'string'})
ozone_df ['FIPS'] = ozone_df['statefips'].astype(str) + ozone_df['countyfips']
# Convert FIPS back to int
ozone_df = ozone_df.astype({'FIPS':'int'})
ozone_df.head()

,Year,Month,Day,statefips,countyfips,O3_max_pred,O3_med_pred,O3_mean_pred,O3_pop_pred,FIPS
0,2001,JAN,1,1,1,31.939058,31.691988,31.680859,31.671226,11
1,2001,JAN,1,1,3,33.646855,33.170271,32.994775,32.935070,13
2,2001,JAN,1,1,5,34.288917,34.068507,34.077954,34.086631,15
3,2001,JAN,1,1,7,30.349767,30.036093,29.931756,29.991733,17
4,2001,JAN,1,1,9,26.472655,25.776595,25.857571,25.872472,19


In [19]:
#aggreagate ozone data by county and year
ozone_df_agg = ozone_df.groupby(['FIPS','Year']).agg({'O3_max_pred':'mean','O3_med_pred':'mean','O3_mean_pred':'mean'})
ozone_df_agg.reset_index(inplace=True)
ozone_df_agg

,FIPS,Year,O3_max_pred,O3_med_pred,O3_mean_pred
0,11,2001,42.305986,41.322810,41.306320
1,11,2002,41.864027,40.669792,40.647115
2,11,2003,40.068504,38.835014,38.825136
3,11,2004,39.985884,38.801989,38.819420
4,11,2005,41.686453,40.529218,40.518377
...,...,...,...,...,...
42345,55141,2010,38.814337,37.804517,37.833975
42346,55141,2011,38.544607,37.451718,37.504409
42347,55141,2012,39.553900,38.588405,38.565205
42348,55141,2013,39.078572,38.264527,38.260503


In [22]:
df = pd.merge(df_cancer, ozone_df_agg, how='inner', left_on=' FIPS', right_on='FIPS')
df

,index,County,FIPS,"Age-Adjusted Incidence Rate(Ê) - cases per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,Average Annual Count,Recent Trend,Recent 5-Year Trend (ˆ) in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1,FIPS,Year,O3_max_pred,O3_med_pred,O3_mean_pred
0,51,"Montgomery County, Alabama(6,10)",1101,61,56.5,65.6,144,falling,-9.6,-15.1,-3.8,1101,2001,43.163842,41.141718,41.197579
1,51,"Montgomery County, Alabama(6,10)",1101,61,56.5,65.6,144,falling,-9.6,-15.1,-3.8,1101,2002,42.436804,40.369983,40.388893
2,51,"Montgomery County, Alabama(6,10)",1101,61,56.5,65.6,144,falling,-9.6,-15.1,-3.8,1101,2003,40.587035,38.347670,38.403803
3,51,"Montgomery County, Alabama(6,10)",1101,61,56.5,65.6,144,falling,-9.6,-15.1,-3.8,1101,2004,41.053774,38.756714,38.853101
4,51,"Montgomery County, Alabama(6,10)",1101,61,56.5,65.6,144,falling,-9.6,-15.1,-3.8,1101,2005,42.718192,40.567937,40.599320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14667,3117,"Wood County, Wisconsin(6,10)",55141,57.4,50.9,64.6,58,stable,-0.7,-10.8,10.5,55141,2010,38.814337,37.804517,37.833975
14668,3117,"Wood County, Wisconsin(6,10)",55141,57.4,50.9,64.6,58,stable,-0.7,-10.8,10.5,55141,2011,38.544607,37.451718,37.504409
14669,3117,"Wood County, Wisconsin(6,10)",55141,57.4,50.9,64.6,58,stable,-0.7,-10.8,10.5,55141,2012,39.553900,38.588405,38.565205
14670,3117,"Wood County, Wisconsin(6,10)",55141,57.4,50.9,64.6,58,stable,-0.7,-10.8,10.5,55141,2013,39.078572,38.264527,38.260503


In [30]:
# how did you get the nulls?
